# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-3.11,84,88,4.36,GS,1690349014
1,1,jamestown,42.0970,-79.2353,20.95,98,0,1.54,US,1690349014
2,2,pisco,-13.7000,-76.2167,16.03,82,0,1.03,PE,1690349014
3,3,impfondo,1.6381,18.0667,22.36,94,100,1.17,CG,1690349014
4,4,kuragino,53.8975,92.6725,20.64,80,100,0.97,RU,1690349015


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color="City",
    size="Humidity",
    frame_width=800,
    frame_height=600,
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
df = city_data_df[(city_data_df["Max Temp"] > 25) & (city_data_df["Humidity"] > 50) & (city_data_df["Cloudiness"] < 30)]

# Drop any rows with null values
# this seems redundant - wasn't this done when we skipped cities early in WeatherPy?
df = df.dropna(how="any")

# Display sample data
df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,atafu village,-8.5421,-172.5159,28.89,74,8,8.11,TK,1690349016
77,77,fale old settlement,-9.3852,-171.2468,28.58,74,7,7.72,TK,1690349020
103,103,deridder,30.8463,-93.2890,27.21,83,0,2.06,US,1690349022
117,117,saint-pierre,-21.3393,55.4781,25.87,53,0,3.09,RE,1690348889
121,121,jamaame,0.0697,42.7450,26.27,67,14,6.23,SO,1690349023


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,atafu village,TK,-8.5421,-172.5159,74,
77,fale old settlement,TK,-9.3852,-171.2468,74,
103,deridder,US,30.8463,-93.2890,83,
117,saint-pierre,RE,-21.3393,55.4781,53,
121,jamaame,SO,0.0697,42.7450,67,
123,siwa oasis,EG,29.2041,25.5195,56,
131,georgetown,MY,5.4112,100.3354,70,
135,key largo,US,25.0865,-80.4473,69,
136,constantia,RO,44.1833,28.6500,67,
145,russellville,US,35.2784,-93.1338,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey" : geoapify_key,
    "categories" : "accommodation.hotel",
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


atafu village - nearest hotel: No hotel found
fale old settlement - nearest hotel: No hotel found
deridder - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
jamaame - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
georgetown - nearest hotel: Page 63 hostel
key largo - nearest hotel: Key Largo Inn
constantia - nearest hotel: Hotel Chérica
russellville - nearest hotel: Holiday Inn
eyl - nearest hotel: فندق ايل دابي
uturoa - nearest hotel: Hawaiki Nui hotel
ceelbuur - nearest hotel: No hotel found
al bawiti - nearest hotel: Old Oasis Hotel
sarangani - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
kamaishi - nearest hotel: 釜石ベイシティホテル
kokopo - nearest hotel: Hotel Kokopo
saint-joseph - nearest hotel: PALM Hotel & Spa
brookings - nearest hotel: Quality Inn
point fortin - nearest hotel: Southwestern Courts
lamu - nearest hotel: Hotel Archipelago villa bed and breakfast
san patricio - nearest hotel: No hotel found
al b

,City,Country,Lat,Lng,Humidity,Hotel Name
24,atafu village,TK,-8.5421,-172.5159,74,No hotel found
77,fale old settlement,TK,-9.3852,-171.2468,74,No hotel found
103,deridder,US,30.8463,-93.2890,83,No hotel found
117,saint-pierre,RE,-21.3393,55.4781,53,Tropic Hotel
121,jamaame,SO,0.0697,42.7450,67,No hotel found
123,siwa oasis,EG,29.2041,25.5195,56,فندق الكيلانى
131,georgetown,MY,5.4112,100.3354,70,Page 63 hostel
135,key largo,US,25.0865,-80.4473,69,Key Largo Inn
136,constantia,RO,44.1833,28.6500,67,Hotel Chérica
145,russellville,US,35.2784,-93.1338,59,Holiday Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    color="City",
    line_color="black",
    size=80,
    frame_width=800,
    frame_height=600,
    hover_cols=["Hotel Name", "Country", "Humidity"]
)

# Display the map
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Humidity)